# Deploying ML model to AWS ☁️

Steps:

1. Go to AWS and create EC2 ubuntu instance. Allow traffic from everywhere. Tick 3 boxes.
2. Open putty and copy instance id info (Click on instance id => Connect => SSH Client)
3. Launch instance.  ( pwd => /home/ubuntu )

4. Update and install python (1:25):
sudo apt-get update
sudo apt-get install python3-venv

Create prod folder and inside it endpoint:
mkdir prod => cd prod => mkdir endpoint

`Inside endpoint` is where we will have our pickle model (kmeans_model.pkl)

Filezilla:
1. File => Site Manager
2. Connect with instance id
3. Get model and drag to folder id



In [23]:
import pickle
import numpy as np

In [24]:
# Read model
with open("kmeans_model.pkl", "rb") as archivo_entrada:
    pickle_kmeans = pickle.load(archivo_entrada)

pickle_kmeans

KMeans(n_clusters=2, n_init=10, random_state=42)

In [25]:
feature_names = ['Interests', 'Shower', 'WashingMachine', 'DishWasher', 'EcoMode',
       'WaterRecycling', 'GeneralRecycling', 'Meat', 'Gasoline', 'Age_Old',
       'Age_Young', 'Estimation_No', 'Estimation_Yes', 'Pet_No', 'Pet_Yes',
       'Garden_No', 'Garden_Yes']

In [29]:
type_0 = np.array([0, 0, 0, 1 ,0, 1, 1, 1, 1, True, False, True, False, True, False, False, True]).reshape(1, -1)
type_1 = np.array([2, 1 ,0, 0, 0, 0, 0, 1, 0, False, True, True, False, False, True, True, False]).reshape(1, -1)


# Set sample as df to prevent error.
sample_survey_df_0 = pd.DataFrame(type_0, columns=feature_names)
sample_survey_df_1 = pd.DataFrame(type_1, columns=feature_names)

# Give me a prediction of 0
print(pickle_kmeans.predict(sample_survey_df_0))
# Give me a prediction of 1
print(pickle_kmeans.predict(sample_survey_df_1))


[1]
[0]


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Create a sample dataset
data = {
    'x': [1, 1.5, 3, 5, 3.5, 4.5, 3.5],
    'y': [1, 2, 4, 7, 5, 5, 4.5]
}
df = pd.DataFrame(data)

# Create and fit the KMeans model
kmeans = KMeans(n_clusters=2,n_init=10, random_state=42)
kmeans.fit(df)

# Get the cluster centers
centers = kmeans.cluster_centers_

# Assuming the feature names are ['x', 'y']
feature_names = ['x', 'y']

# Print the cluster centers with feature names
for center in centers:
    for feature, value in zip(feature_names, center):
        print(f"{feature}: {value}")
    print()

print(feature_names)

cluster_centers_df = pd.DataFrame(centers, columns=feature_names)
cluster_centers_df

x: 3.9
y: 5.1

x: 1.25
y: 1.5

['x', 'y']


,x,y
0,3.90,5.1
1,1.25,1.5
